In [1]:
from langchain_openai import ChatOpenAI
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent
from langchain.tools.tavily_search import TavilySearchResults
from langchain.tools import tool
from collections import defaultdict
from dotenv import load_dotenv
from typing import List
import requests
import re

In [2]:
load_dotenv()
LLM = ChatOpenAI(model = "gpt-4.1")
search = TavilySearchResults() # will be used to search the web

In [3]:
# List containing IPL ids of different years -> used in multiple functions
IPL_IDs = []
# Manually define the year-to-ID mapping
year_to_id = {
    "2008": "2058",
    "2009": "2059",
    "2010": "2060",
    "2011": "2037",
    "2012": "2115",
    "2013": "2170",
    "2014": "2261",
    "2015": "2330",
    "2016": "2430",
    "2017": "2568",
    "2018": "2676",
    "2019": "2810",
    "2020": "3130",
    "2021": "3472",
    "2022": "4061",
    "2023": "5945",
    "2024": "7607",
    "2025":"9237"
}

# Create the list of dictionaries
for year, league_id in year_to_id.items():
    league_name = f"Indian Premier League {year}"
    IPL_IDs.append({league_name: league_id})

# Example output
for league in IPL_IDs:
    print(league)

{'Indian Premier League 2008': '2058'}
{'Indian Premier League 2009': '2059'}
{'Indian Premier League 2010': '2060'}
{'Indian Premier League 2011': '2037'}
{'Indian Premier League 2012': '2115'}
{'Indian Premier League 2013': '2170'}
{'Indian Premier League 2014': '2261'}
{'Indian Premier League 2015': '2330'}
{'Indian Premier League 2016': '2430'}
{'Indian Premier League 2017': '2568'}
{'Indian Premier League 2018': '2676'}
{'Indian Premier League 2019': '2810'}
{'Indian Premier League 2020': '3130'}
{'Indian Premier League 2021': '3472'}
{'Indian Premier League 2022': '4061'}
{'Indian Premier League 2023': '5945'}
{'Indian Premier League 2024': '7607'}
{'Indian Premier League 2025': '9237'}


In [6]:
from pprint import pprint

pprint(search.invoke({"query": "is virat kohli available for today's ipl match vs LSG"}))


[{'content': 'The Economic Times daily newspaper is available online now.\n'
             '\n'
             '\n'
             '\n'
             'LSG vs RCB Key Players: From Digvesh Singh to Virat Kohli, Josh '
             "Hazlewood, here are top names who will shape today's clash\n"
             '\n'
             'Royal Challengers Bengaluru faces Lucknow Super Giants in a '
             'crucial IPL clash, aiming for a top-two finish. RCB confronts '
             "LSG's formidable batting lineup, despite Aiden Markram's absence "
             'due to international duties. The partnership of Markram and '
             'Marsh has been prolific, amassing significant runs.\n'
             '\n'
             'Live Events [...] Uh-oh! This is an exclusive story available '
             'for selected readers only.\n'
             '\n'
             'Worry not. You’re just a step away.\n'
             '\n'
             'Prime Account Detected!\n'
             '\n'
             "It seems like

In [ ]:
# TOOLS
@tool
def match_info():
    url = "https://Cricbuzz-Official-Cricket-API.proxy-production.allthingsdev.co/matches/upcoming"
    headers = {
        'x-apihub-key': '9HN92wz6l7bberNNuKkhDCXeb4YH4lXo2fIKuVdgCpB82jpHlM', # API KEY
        'x-apihub-host': 'Cricbuzz-Official-Cricket-API.allthingsdev.co',
        'x-apihub-endpoint': '1943a818-98e9-48ea-8d1c-1554e116ef44'
    }

    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        raise Exception(f"API request failed: {response.status_code}")
    
    data = response.json()
    ipl_match_list = []

    for type_match in data.get("typeMatches", []):
        for series_match in type_match.get("seriesMatches", []):
            series = series_match.get("seriesAdWrapper", {})
            if "Indian Premier League" in series.get("seriesName", ""):
                for match in series.get("matches", []):
                    match_info = match.get("matchInfo", {})
                    match_id = match_info.get("matchId")
                    match_desc = match_info.get("matchDesc")
                    match_status = match_info.get("status")
                    team1 = match_info.get("team1", {}).get("teamName", "Team 1")
                    team2 = match_info.get("team2", {}).get("teamName", "Team 2")
                    venue = match_info.get("venueInfo", {})
                    venue_id = venue.get("id", "Unknown ID")
                    ground = venue.get("ground", "Unknown Ground")
                    city = venue.get("city", "Unknown City")

                    ipl_match_list.append({
                        "Match ID": match_id,
                        "Match Desc": match_desc,
                        "Teams": f"{team1} vs {team2}",
                        "Status": match_status,
                        "Venue ID": venue_id,
                        "Venue": f"{ground}, {city}"
                    })
    
    return ipl_match_list


@tool
def additional_info(match_id: str) -> str: # about pitch, probable players, injuries about ground,
    """zxcv"""
    url = f"https://Cricbuzz-Official-Cricket-API.proxy-production.allthingsdev.co/match/{match_id}/commentary"
    headers = {
        'x-apihub-key': '9HN92wz6l7bberNNuKkhDCXeb4YH4lXo2fIKuVdgCpB82jpHlM',
        'x-apihub-host': 'Cricbuzz-Official-Cricket-API.allthingsdev.co',
        'x-apihub-endpoint': '8cb69a0f-bcaa-45b5-a016-229a2e7594f6'
    }

    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        raise Exception(f"Failed to get commentary: {response.status_code}")

    data = response.json()
    full_text = ""

    # Concatenate all commText entries
    for item in data.get("commentaryList", []):
        full_text += item.get("commText", "") + " "

    # Remove ALL markers like B0$, B1$, B14$ (anywhere in the text)
    cleaned = re.sub(r'\s*B\d+\$', '', full_text)
    
    # Remove escaped newlines and excess spaces
    cleaned = cleaned.replace("\\n", " ")
    cleaned = re.sub(r'\s+', ' ', cleaned).strip()

    return cleaned

In [9]:
# 1. Researcher Agent: gathers match details (teams, date, venue, weather, pitch, odds).
research_agent = create_react_agent(
    model = LLM,
    name = "researcher",
    tools = [search, match_info, additional_info],
    prompt = (
        """
            You are a cricket expert assistant. When asked about an upcoming IPL match, first use the `match_info` tool to identify the match, 
            teams, venue, status and match ID. Then use the `additional_info` tool with the match ID to extract details like pitch report, 
            probable playing XI, injuries, and venue records. Present all the details clearly and effeciently.
        """
    )
)

In [10]:
inputs = {"messages": [{"role": "user", "content": "Give me info about today's ipl match"}]}
result = research_agent.invoke(inputs)
for r in result['messages']:
    print(r)
print(result["messages"][-1].content)

content="Give me info about today's ipl match" additional_kwargs={} response_metadata={} id='3e6d7306-1ced-48ac-baa5-f2f76cef11e6'
content='' additional_kwargs={'tool_calls': [{'id': 'call_sUJPwYcoqOpT9PLzRmeKotnG', 'function': {'arguments': '{}', 'name': 'match_info'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 205, 'total_tokens': 215, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_51e1070cf2', 'id': 'chatcmpl-BbmU0O83thmbykBIkJLOH60LSJt2E', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} name='researcher' id='run--13efa8c8-25c9-4d21-b5ef-468036e4f38c-0' tool_calls=[{'name': 'match_info', 'args': {}, 'id': 'call_sUJPwYcoqOpT9PLzRmeKotnG', 'type': 'tool_call'}] usage_m

In [ ]:
# TOOLS -> DATA COLLECTOR AGENT
@tool
def player_id_search(player_names: List[str]) -> List[dict]:        # get the player id and name for a list of player names
    """This tool returns the player ID and name for a list of player names.
        It takes a list containing names of players as input and output a list 
        of dict with each dict containing the player id and name of that player."""
    
    headers = {
        'x-apihub-key': '9HN92wz6l7bberNNuKkhDCXeb4YH4lXo2fIKuVdgCpB82jpHlM',
        'x-apihub-host': 'Cricbuzz-Official-Cricket-API.allthingsdev.co',
        'x-apihub-endpoint': 'b0242771-45ea-4c07-be42-a6da38cdec41'
    }

    results = []

    for name in player_names:
        url = f"https://Cricbuzz-Official-Cricket-API.proxy-production.allthingsdev.co/browse/player?search={name.replace(' ', '+')}"

        response = requests.get(url, headers=headers)

        if response.status_code != 200:
            results.append({"player name": name, "error": f"API call failed with status code {response.status_code}"})
            continue

        data = response.json()
        players = data.get("player", [])
        if not players:
            results.append({"player name": name, "error": "No player found"})
            continue

        player_id = players[0].get("id")
        player_name = players[0].get("name")

        if not player_id:
            results.append({"player name": name, "error": "Player ID not found"})
            continue

        results.append({
            "player name": player_name,
            "player id": str(player_id)
        })

    return results

In [120]:
@tool
def player_ipl_stats(player_ids: List[str]) -> List[dict]: # Overall ipl stats
    """It fetches IPL batting and bowling stats for a list of player IDs.
    It returns stats based on player role:
    - Batter: only batting stats
    - Bowler: only bowling stats
    - All-rounder: both
    """

    def fetch_stats(url: str, headers: dict) -> dict:
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            return {}
        return response.json()

    def extract_ipl_stats(data: dict) -> dict:
        headers_list = data.get("headers", [])
        values_list = data.get("values", [])

        try:
            ipl_index = headers_list.index("IPL")
        except ValueError:
            return {}

        stats = {}
        for item in values_list:
            row = item.get("values", [])
            if len(row) > ipl_index:
                stat_name = row[0]
                stat_value = row[ipl_index]
                stats[stat_name] = stat_value
        return stats

    results = []

    for player_id in player_ids:
        # Fetch role info first
        role_url = f"https://Cricbuzz-Official-Cricket-API.proxy-production.allthingsdev.co/browse/player/{player_id}"
        role_headers = {
            'x-apihub-key': '9HN92wz6l7bberNNuKkhDCXeb4YH4lXo2fIKuVdgCpB82jpHlM',
            'x-apihub-host': 'Cricbuzz-Official-Cricket-API.allthingsdev.co',
            'x-apihub-endpoint': 'a055bf38-0796-4fab-8fe3-6f042f04cdba'
        }
        role_info = fetch_stats(role_url, role_headers)
        player_role = role_info.get("role", "Unknown").lower()

        name_url = f"https://Cricbuzz-Official-Cricket-API.proxy-production.allthingsdev.co/browse/player/{player_id}"

        name_headers = {
        'x-apihub-key': 'OCJkYg864FiHU7sYIp2YOpCsUtNcvA35gc88LpOBbtY8c2enSf',
        'x-apihub-host': 'Cricbuzz-Official-Cricket-API.allthingsdev.co',
        'x-apihub-endpoint': 'a055bf38-0796-4fab-8fe3-6f042f04cdba'
        }
        name_info = fetch_stats(name_url, name_headers)
        player_name = name_info.get("name", "Unknown")       

        player_result = {
            "Name": player_name,
            "ID": player_id,
            "Role": player_role.title(),
        }

        # Fetch batting if applicable
        if "batsman" in player_role or "allrounder" in player_role:
            batting_url = f"https://Cricbuzz-Official-Cricket-API.proxy-production.allthingsdev.co/browse/player/{player_id}/batting"
            batting_headers = {
                'x-apihub-key': '9HN92wz6l7bberNNuKkhDCXeb4YH4lXo2fIKuVdgCpB82jpHlM',
                'x-apihub-host': 'Cricbuzz-Official-Cricket-API.allthingsdev.co',
                'x-apihub-endpoint': '07a4d9b5-092e-4035-adc7-253bc3532a81'
            }
            batting_data = fetch_stats(batting_url, batting_headers)
            ipl_batting_stats = extract_ipl_stats(batting_data)
            player_result["Batting Stats"] = ipl_batting_stats

        # Fetch bowling if applicable
        if "bowler" in player_role or "allrounder" in player_role:
            bowling_url = f"https://Cricbuzz-Official-Cricket-API.proxy-production.allthingsdev.co/browse/player/{player_id}/bowling"
            bowling_headers = {
                'x-apihub-key': '9HN92wz6l7bberNNuKkhDCXeb4YH4lXo2fIKuVdgCpB82jpHlM',
                'x-apihub-host': 'Cricbuzz-Official-Cricket-API.allthingsdev.co',
                'x-apihub-endpoint': '5ba067de-b9a5-446f-916b-9dfbef717211'
            }
            bowling_data = fetch_stats(bowling_url, bowling_headers)
            ipl_bowling_stats = extract_ipl_stats(bowling_data)
            player_result["Bowling Stats"] = ipl_bowling_stats

        results.append(player_result)

    return results


In [121]:

ipl_stats = player_ipl_stats({"player_ids": ["576", "7909", "9647"]})  # innings, affregate functions, balls
for player in ipl_stats:
    print(player)

{'Name': 'Rohit Sharma', 'ID': '576', 'Role': 'Batsman', 'Batting Stats': {'Matches': '270', 'Innings': '265', 'Runs': '6957', 'Balls': '5277', 'Highest': '109', 'Average': '29.6', 'SR': '131.84', 'Not Out': '30', 'Fours': '630', 'Sixes': '298', 'Ducks': '18', '50s': '46', '100s': '2', '200s': '0', '300s': '0', '400s': '0'}}
{'Name': 'Mohammed Shami', 'ID': '7909', 'Role': 'Bowler', 'Bowling Stats': {'Matches': '119', 'Innings': '119', 'Balls': '2606', 'Runs': '3748', 'Maidens': '3', 'Wickets': '133', 'Avg': '28.18', 'Eco': '8.63', 'SR': '19.59', 'BBI': '4/11', 'BBM': '4/11', '4w': '2', '5w': '0', '10w': '0'}}
{'Name': 'Hardik Pandya', 'ID': '9647', 'Role': 'Batting Allrounder', 'Batting Stats': {'Matches': '150', 'Innings': '138', 'Runs': '2712', 'Balls': '1849', 'Highest': '91', 'Average': '28.25', 'SR': '146.68', 'Not Out': '42', 'Fours': '206', 'Sixes': '145', 'Ducks': '6', '50s': '10', '100s': '0', '200s': '0', '300s': '0', '400s': '0'}, 'Bowling Stats': {'Matches': '150', 'Inning

In [85]:
@tool
def get_team_id(series_id: str) -> List[dict]:
    """dcftv"""
    url = f"https://Cricbuzz-Official-Cricket-API.proxy-production.allthingsdev.co/series/{series_id}/squads"

    headers = {
        'x-apihub-key': 'OCJkYg864FiHU7sYIp2YOpCsUtNcvA35gc88LpOBbtY8c2enSf',
        'x-apihub-host': 'Cricbuzz-Official-Cricket-API.allthingsdev.co',
        'x-apihub-endpoint': '038d223b-aca5-4096-8eb1-184dd0c09513'
    }

    response = requests.get(url, headers=headers)
    data = response.json()

    # Extract relevant team info from squads
    team_list = []
    for squad in data.get("squads", []):
        if "teamId" in squad and "squadType" in squad:
            team_list.append({
                "teamName": squad["squadType"],
                "teamId": squad["teamId"]
            })

    return team_list 

In [93]:
@tool
def get_recent_match_ids(series_id: str, team_ids: List[str]) -> List[dict]:
    """Get up to 5 most recent match IDs for a team in a given series (IPL season)."""

    url = f"https://Cricbuzz-Official-Cricket-API.proxy-production.allthingsdev.co/series/{series_id}"
    headers = {
        'x-apihub-key': 'OCJkYg864FiHU7sYIp2YOpCsUtNcvA35gc88LpOBbtY8c2enSf',
        'x-apihub-host': 'Cricbuzz-Official-Cricket-API.allthingsdev.co',
        'x-apihub-endpoint': '661c6b89-b558-41fa-9553-d0aca64fcb6f'
    }

    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        raise Exception(f"Failed to fetch series data: {response.status_code}")
    
    # Convert team_ids to integers for correct comparison
    team_ids = [int(team_id) for team_id in team_ids]

    data = response.json()
    match_entries = data.get("matchDetails", [])
    
    # Dictionary to collect matches for each team
    team_matches = {team_id: [] for team_id in team_ids}

    for entry in match_entries:
        day_matches = entry.get("matchDetailsMap", {}).get("match", [])
        for match in day_matches:
            info = match.get("matchInfo", {})
            if info.get("state") != "Complete":
                continue

            t1 = info.get("team1", {}).get("teamId")
            t2 = info.get("team2", {}).get("teamId")
            match_id = info.get("matchId")
            start_date = int(info.get("startDate", 0))

            for team_id in team_ids:
                if team_id == t1 or team_id == t2:
                    team_matches[team_id].append({
                        "matchId": match_id,
                        "startDate": start_date
                    })

    # Format result: sort each list by date and pick top 5
    result = []
    for team_id in team_ids:
        sorted_matches = sorted(team_matches[team_id], key=lambda x: x["startDate"], reverse=True)
        recent_ids = [m["matchId"] for m in sorted_matches[:5]]
        result.append({
            "teamId": team_id,
            "matchIds": recent_ids
        })

    return result

In [126]:
@tool
def get_team_player_stats(team_id: str, match_ids: List[str]):
    """Fetch and aggregate player stats (batting and bowling) for a team across multiple matches, filtered by role."""

    player_stats = defaultdict(lambda: {
        'player_name': '',
        'player_id': 0,
        'role': None,
        'is_wicketkeeper': False,
        'is_overseas': False,
        'Batting Stats': {
            'runs': 0,
            'balls': 0,
            '4s': 0,
            '6s': 0,
            'strike_rate': 0,
            'average': 0,
            'innings': 0,
            'notouts': 0,
        },
        'Bowling Stats': {
            'runs_conceded': 0,
            'balls_bowled': 0,
            'dot_balls': 0,
            'wicket': 0,
            'economy': 0,
        }
    })

    def fetch_stats(url: str, headers: dict) -> dict:
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            return {}
        return response.json()

    for match_id in match_ids:
        scorecard_url = f"https://Cricbuzz-Official-Cricket-API.proxy-production.allthingsdev.co/match/{match_id}/scorecard"
        scorecard_headers = {
            'x-apihub-key': 'OCJkYg864FiHU7sYIp2YOpCsUtNcvA35gc88LpOBbtY8c2enSf',
            'x-apihub-host': 'Cricbuzz-Official-Cricket-API.allthingsdev.co',
            'x-apihub-endpoint': '5f260335-c228-4005-9eec-318200ca48d6'
        }

        data = fetch_stats(scorecard_url, scorecard_headers)
        scorecards = data.get("scoreCard", [])

        for innings in scorecards:
            # Batting
            bat_team = innings.get("batTeamDetails", {})
            if str(bat_team.get("batTeamId")) == str(team_id):
                for batsman in bat_team.get("batsmenData", []):
                    pid = batsman["batId"]
                    player = player_stats[pid]
                    player['player_name'] = batsman["batName"]
                    player['player_id'] = pid
                    player['is_wicketkeeper'] |= batsman.get("isKeeper", False)
                    player['is_overseas'] |= batsman.get("isOverseas", False)

                    player['Batting Stats']['runs'] += batsman.get("runs", 0)
                    player['Batting Stats']['balls'] += batsman.get("balls", 0)
                    player['Batting Stats']['4s'] += batsman.get("fours", 0)
                    player['Batting Stats']['6s'] += batsman.get("sixes", 0)
                    player['Batting Stats']['innings'] += 1
                    if batsman.get("outDesc", "").lower() == "not out":
                        player['Batting Stats']['notouts'] += 1

            # Bowling
            bowl_team = innings.get("bowlTeamDetails", {})
            if str(bowl_team.get("bowlTeamId")) == str(team_id):
                for bowler in bowl_team.get("bowlersData", []):
                    pid = bowler["bowlerId"]
                    player = player_stats[pid]
                    player['player_name'] = bowler["bowlName"]
                    player['player_id'] = pid
                    player['is_wicketkeeper'] |= bowler.get("isKeeper", False)
                    player['is_overseas'] |= bowler.get("isOverseas", False)

                    player['Bowling Stats']['balls_bowled'] += bowler.get("balls", 0)
                    player['Bowling Stats']['wicket'] += bowler.get("wickets", 0)
                    player['Bowling Stats']['dot_balls'] += bowler.get("dotBalls", 0)

    final_stats = []

    for pid, player in player_stats.items():
        bat = player['Batting Stats']
        bowl = player['Bowling Stats']

        # Fetch role
        role_url = f"https://Cricbuzz-Official-Cricket-API.proxy-production.allthingsdev.co/browse/player/{pid}"
        role_headers = {
            'x-apihub-key': 'OCJkYg864FiHU7sYIp2YOpCsUtNcvA35gc88LpOBbtY8c2enSf',
            'x-apihub-host': 'Cricbuzz-Official-Cricket-API.allthingsdev.co',
            'x-apihub-endpoint': 'a055bf38-0796-4fab-8fe3-6f042f04cdba'
        }
        role_info = fetch_stats(role_url, role_headers)
        role = role_info.get("role", "Unknown").lower()
        player['role'] = role.title()

        # Batting stats calculations
        if bat['balls'] > 0:
            bat['strike_rate'] = round((bat['runs'] / bat['balls']) * 100, 2)
        outs = bat['innings'] - bat['notouts']
        if outs > 0:
            bat['average'] = round(bat['runs'] / outs, 2)

        # Bowling stats calculations
        if bowl['balls_bowled'] > 0:
            overs = bowl['balls_bowled'] / 6
            bowl['economy'] = round(bowl['runs_conceded'] / overs, 2)

        # Filter stats by role
        filtered_player = {
            "player_id": player['player_id'],
            "player_name": player['player_name'],
            "role": player['role'],
            "is_wicketkeeper": player['is_wicketkeeper'],
            "is_overseas": player['is_overseas']
        }

        if "batter" in role:
            filtered_player["Batting Stats"] = bat
        elif "bowler" in role:
            filtered_player["Bowling Stats"] = bowl
        elif "allrounder" in role:
            filtered_player["Batting Stats"] = bat
            filtered_player["Bowling Stats"] = bowl
        else:
            # Optional: include both for unknowns
            filtered_player["Batting Stats"] = bat
            filtered_player["Bowling Stats"] = bowl

        final_stats.append(filtered_player)

    return final_stats


In [127]:

ipl_stats = get_team_player_stats({"team_id": "971", "match_ids": ["118880", "118862"]})  # innings, affregate functions, balls
for player in ipl_stats:
    print(player)

{'player_id': 10808, 'player_name': 'Mohammed Siraj', 'role': 'Bowler', 'is_wicketkeeper': False, 'is_overseas': False, 'Bowling Stats': {'runs_conceded': 0, 'balls_bowled': 80, 'dot_balls': 0, 'wicket': 0, 'economy': 0.0}}
{'player_id': 18637, 'player_name': 'Arshad Khan', 'role': 'Bowling Allrounder', 'is_wicketkeeper': False, 'is_overseas': False, 'Batting Stats': {'runs': 21, 'balls': 17, '4s': 0, '6s': 3, 'strike_rate': 123.53, 'average': 10.5, 'innings': 2, 'notouts': 0}, 'Bowling Stats': {'runs_conceded': 0, 'balls_bowled': 50, 'dot_balls': 0, 'wicket': 1, 'economy': 0.0}}
{'player_id': 10551, 'player_name': 'Prasidh Krishna', 'role': 'Bowler', 'is_wicketkeeper': False, 'is_overseas': False, 'Bowling Stats': {'runs_conceded': 0, 'balls_bowled': 80, 'dot_balls': 0, 'wicket': 2, 'economy': 0.0}}
{'player_id': 13320, 'player_name': 'Gerald Coetzee', 'role': 'Bowler', 'is_wicketkeeper': False, 'is_overseas': True, 'Bowling Stats': {'runs_conceded': 0, 'balls_bowled': 30, 'dot_balls'

## Create function for finding player id(from name) -> last 5 match of T20 format , last 5 match againts the oppostion, last 5 match on the venue (avg,highest,SR)

In [ ]:
# 2. Data Collector Agent: 
## collects player performance stats (last 5 matches - total, versus team, versus venue) , overall performance versus oppostion bowlers
data_collector_agent = create_react_agent(
    model = LLM,
    name = "data_miner",
    tools = [search, player_id_search,player_ipl_stats],
    prompt = (
        "You are an agent which is phenomenal at maths. Answer the query of the user to the best you can."
    )
)

In [ ]:
from typing import Dict

@tool
def batsman_stats_comparator(players: List[Dict]) -> str:
    """
    Compare and rank batsmen based on T20 stats:
    - Recent form (avg, SR, HS),
    - Performance vs opposition (avg, SR, HS),
    - Venue performance (avg, SR, HS),
    - Optionally: dismissals by bowlers.
    """
    def score_batsman(p):
        score = 0
        score += p["recent"]["avg"] * 0.4 + p["recent"]["sr"] * 0.3 + p["recent"]["hs"] * 0.3
        score += p["vs_opposition"]["avg"] * 0.3 + p["vs_opposition"]["sr"] * 0.3 + p["vs_opposition"]["hs"] * 0.4
        score += p["at_venue"]["avg"] * 0.3 + p["at_venue"]["sr"] * 0.3 + p["at_venue"]["hs"] * 0.4
        if "dismissals" in p:
            score -= sum(p["dismissals"].values()) * 5  # Penalty per dismissal
        return score

    ranked = sorted(players, key=score_batsman, reverse=True)     ## sort the batsmen based on their score in descending order
    result = "🏏 Batsman Ranking:\n"
    for i, p in enumerate(ranked):
        reason = f'{p["name"]}: Strong recent avg {p["recent"]["avg"]}, SR {p["recent"]["sr"]}, HS {p["recent"]["hs"]}. '
        reason += f'Good vs opposition and at venue. '
        if "dismissals" in p:
            reason += f'Lost wicket to key bowlers {list(p["dismissals"].keys())}. '
        result += f"{i+1}. {reason}\n"
    return result


In [ ]:
@tool
def bowler_stats_comparator(players: List[Dict]) -> str:
    """
    Compare and rank bowlers based on:
    - Recent form (avg wickets, econ, best),
    - Vs opposition,
    - At venue.
    """
    def score_bowler(p):
        score = 0
        score += p["recent"]["avg_wkts"] * 0.5 - p["recent"]["econ"] * 0.3 + p["recent"]["best"] * 0.2
        score += p["vs_opposition"]["avg_wkts"] * 0.4 - p["vs_opposition"]["econ"] * 0.3 + p["vs_opposition"]["best"] * 0.3
        score += p["at_venue"]["avg_wkts"] * 0.4 - p["at_venue"]["econ"] * 0.3 + p["at_venue"]["best"] * 0.3
        return score

    ranked = sorted(players, key=score_bowler, reverse=True)
    result = "🎯 Bowler Ranking:\n"
    for i, p in enumerate(ranked):
        reason = f'{p["name"]}: Recent wickets {p["recent"]["avg_wkts"]}, Econ {p["recent"]["econ"]}, Best {p["recent"]["best"]}. '
        reason += f'Strong vs opposition and at venue.'
        result += f"{i+1}. {reason}\n"
    return result


In [ ]:
@tool
def allrounder_stats_comparator(players: List[Dict]) -> str:
    """
    Compare all-rounders using combined batting + bowling metrics.
    """
    def score_allrounder(p):
        bat = p["batting"]
        bowl = p["bowling"]
        bat_score = (
            bat["recent"]["avg"] * 0.3 + bat["recent"]["sr"] * 0.2 + bat["recent"]["hs"] * 0.2 +
            bat["vs_opposition"]["avg"] * 0.15 + bat["at_venue"]["avg"] * 0.15
        )
        bowl_score = (
            bowl["recent"]["avg_wkts"] * 0.3 - bowl["recent"]["econ"] * 0.2 + bowl["recent"]["best"] * 0.1 +
            bowl["vs_opposition"]["avg_wkts"] * 0.2 + bowl["at_venue"]["avg_wkts"] * 0.2
        )
        return bat_score + bowl_score

    ranked = sorted(players, key=score_allrounder, reverse=True)
    result = "🔁 All-Rounder Ranking:\n"
    for i, p in enumerate(ranked):
        reason = f'{p["name"]}: Good balance with avg {p["batting"]["recent"]["avg"]}, '
        reason += f'wickets {p["bowling"]["recent"]["avg_wkts"]}, economy {p["bowling"]["recent"]["econ"]}.'
        result += f"{i+1}. {reason}\n"
    return result


In [ ]:
from langchain.tools import tool

@tool
def unified_player_comparator(players: List[Dict]) -> List[Dict]:
    """
    Rank all players (batsmen, bowlers, all-rounders) based on role-specific stats.
    Returns a list of player objects with name, role, score, and reasoning.
    """
    def score_batsman(p):
        score = 0
        score += p["recent"]["avg"] * 0.4 + p["recent"]["sr"] * 0.3 + p["recent"]["hs"] * 0.3
        score += p["vs_opposition"]["avg"] * 0.3 + p["vs_opposition"]["sr"] * 0.3 + p["vs_opposition"]["hs"] * 0.4
        score += p["at_venue"]["avg"] * 0.3 + p["at_venue"]["sr"] * 0.3 + p["at_venue"]["hs"] * 0.4
        if "dismissals" in p:
            score -= sum(p["dismissals"].values()) * 5
        return score

    def score_bowler(p):
        score = 0
        score += p["recent"]["avg_wkts"] * 0.5 - p["recent"]["econ"] * 0.3 + p["recent"]["best"] * 0.2
        score += p["vs_opposition"]["avg_wkts"] * 0.4 - p["vs_opposition"]["econ"] * 0.3 + p["vs_opposition"]["best"] * 0.3
        score += p["at_venue"]["avg_wkts"] * 0.4 - p["at_venue"]["econ"] * 0.3 + p["at_venue"]["best"] * 0.3
        return score

    def score_allrounder(p):
        bat = p["batting"]
        bowl = p["bowling"]
        bat_score = (
            bat["recent"]["avg"] * 0.3 + bat["recent"]["sr"] * 0.2 + bat["recent"]["hs"] * 0.2 +
            bat["vs_opposition"]["avg"] * 0.15 + bat["at_venue"]["avg"] * 0.15
        )
        bowl_score = (
            bowl["recent"]["avg_wkts"] * 0.3 - bowl["recent"]["econ"] * 0.2 + bowl["recent"]["best"] * 0.1 +
            bowl["vs_opposition"]["avg_wkts"] * 0.2 + bowl["at_venue"]["avg_wkts"] * 0.2
        )
        return bat_score + bowl_score

    result = []
    for p in players:
        role = p["role"]
        if role == "batsman":
            score = score_batsman(p)
            reason = (
                f'{p["name"]}: Batsman - recent avg {p["recent"]["avg"]}, SR {p["recent"]["sr"]}, HS {p["recent"]["hs"]}. '
                f'Opposition avg {p["vs_opposition"]["avg"]}, Venue avg {p["at_venue"]["avg"]}.'
            )
            if "dismissals" in p:
                reason += f' Dismissed by {list(p["dismissals"].keys())}.'
        elif role == "bowler":
            score = score_bowler(p)
            reason = (
                f'{p["name"]}: Bowler - recent wkts {p["recent"]["avg_wkts"]}, Econ {p["recent"]["econ"]}, Best {p["recent"]["best"]}. '
                f'Opposition avg {p["vs_opposition"]["avg_wkts"]}, Venue avg {p["at_venue"]["avg_wkts"]}.'
            )
        elif role == "allrounder":
            score = score_allrounder(p)
            reason = (
                f'{p["name"]}: All-rounder - Bat avg {p["batting"]["recent"]["avg"]}, '
                f'Bowl wkts {p["bowling"]["recent"]["avg_wkts"]}, Econ {p["bowling"]["recent"]["econ"]}.'
            )
        else:
            continue  # skip unknown roles

        result.append({
            "name": p["name"],
            "role": role,
            "score": round(score, 2),
            "reason": reason
        })

    result.sort(key=lambda x: x["score"], reverse=True)
    return result


In [ ]:
# Player Comparison agent
## Rank the players based on stats.
player_comparator = create_react_agent(
    model = LLM,
    name = "player_faceoff",
    tools = [batsman_stats_comparator, bowler_stats_comparator,allrounder_stats_comparator,unified_player_comparator],
    prompt = (
        "You are an agent which is phenomenal at maths. Answer the query of the user to the best you can."
    )
)

In [11]:
print(search.invoke({"query": "what is fantasy cricket and how to play it"}))

[{'title': 'What does fantasy cricket mean? How do you play it? - Quora', 'url': 'https://www.quora.com/What-does-fantasy-cricket-mean-How-do-you-play-it', 'content': 'Fantasy cricket allows users to play cricket virtually by selecting players of their own choice. Each player is assigned points basis wickets,', 'score': 0.8282873}, {'title': 'How to Play Fantasy Cricket Like a Pro? - Top Tips to Win Every Game', 'url': 'https://www.gamezy.com/blog/fantasy-cricket/how-to-play-fantasy-cricket/', 'content': 'How to Play Fantasy Cricket?\n\nIf you are looking to play fantasy cricket online, you will have to have the tactical nous and know-how to predict a team of XI players who can win you the most points. Fantasy cricket isn’t about spending more, but about knowing exactly how particular players will perform in a game and taking the right calls before the match begins.\n\n\n\nHow to Play Fantasy Cricket Like a Pro?\n\nTop tips that will help you to play fantasy cricket more efficiently ar

In [15]:
print(search.invoke({"query": "what is average of shubhman gill in last 5 T20 matches of all time"}))

[{'title': 'Shubman Gill - Wikipedia', 'url': 'https://en.wikipedia.org/wiki/Shubman_Gill', 'content': 'Last T20I 30 July 2024 vSri Lanka\nT20I shirt no.77\nDomestic team information\nYears Team\n2017–presentPunjab\n2018–2021Kolkata Knight Riders\n2022–presentGujarat Titans\n2022Glamorgan\nCareer statistics\n| Competition | Test | ODI | T20I | FC |\n| --- | --- | --- | --- | --- |\n| Matches | 32 | 55 | 21 | 61 |\n| Runs scored | 1,893 | 2,775 | 578 | 4,587 |\n| Batting average | 35.05 | 59.04 | 30.42 | 47.28 |\n| 100s/50s | 5/7 | 8/15 | 1/3 | 14/19 |\n| Top score | 128 | 208 | 126* | 268 |', 'score': 0.8127637}, {'title': 'Shubman Gill Profile: Age, Stats, Records, ICC Ranking, Career Info ...', 'url': 'https://www.mykhel.com/cricket/players/shubman-gill-p11023/', 'content': '582 runs, with an average of 31. Shubman Gill has hit 61 fours and 22 sixes in his T20 career.In his most recent T20 match against Sri Lanka at Pallekele', 'score': 0.7940821}, {'title': 'Shubman Gill Profile - C